In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal
import numpy as np
import seaborn as sns

In [2]:
plt.rcParams["figure.figsize"]=[15,5]

In [3]:
raw_pp=pd.read_csv('9thhour.csv', usecols=['timestamp', 'data.ax', 'data.az'])
raw_pp.dropna(axis=1, inplace=True)
raw_pp['timestamp'] =  pd.to_datetime(raw_pp['timestamp'])

In [4]:
#Load LD data and plot
raw_pp["acc"] = ( raw_pp["data.ax"]**2 + raw_pp["data.az"]**2) ** 0.5
acc=raw_pp[["timestamp", "acc"]]
acc.index = acc.timestamp
test_pp=acc
# test_pp['acc_rolling_sum']=pd.Series.to_frame(test_pp.acc.rolling(100*10, center=True).sum())
test_pp['acc_rolling_std']=pd.Series.to_frame(test_pp.acc.rolling(100*2, center=True).std())


/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [5]:
%matplotlib qt

In [6]:
# plt.plot(test_pp['timestamp'], test_pp['acc'])
# plt.plot(test_pp['timestamp'], test_pp['acc_rolling_std'])

In [7]:
test_pp['detections']=pd.Series.to_frame(test_pp.acc_rolling_std.rolling(100, center=True).sum())

In [8]:
plt.plot(test_pp['timestamp'], test_pp['acc']*100)
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [9]:
plt.plot(test_pp['timestamp'], test_pp['detections'])

In [13]:
test_pp['boards']=0
test_pp['boards'][test_pp['detections']>=2] = 1

/home/raghu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
plt.plot(test_pp["timestamp"],test_pp["boards"])

In [15]:
working_times_raw=scipy.signal.find_peaks(test_pp.boards, distance=40*72, width=1)

In [16]:
working_times_raw_df=pd.DataFrame({"sample_number":working_times_raw[0], "working_time":working_times_raw[1]['widths']/72})

In [17]:
working_times_df=working_times_raw_df[working_times_raw_df.working_time>30]

In [18]:
PP_events=pd.DataFrame({"timestamp":test_pp.iloc[working_times_df.sample_number].timestamp, "event":1, "working_time":working_times_df.working_time.tolist()})
PP_events.drop("timestamp",axis=1, inplace=True)

In [19]:
PP_events.working_time.mode().mean()

59.111111111111114

In [21]:
PP_events

,event,working_time
timestamp,,
2019-01-02 09:03:48.801577,1,58.611111
2019-01-02 09:05:41.799734,1,59.152778
2019-01-02 09:09:12.983902,1,58.916667
2019-01-02 09:11:05.031178,1,61.347222
2019-01-02 09:12:57.702438,1,63.055556
2019-01-02 09:14:49.710774,1,64.583333
2019-01-02 09:16:34.976179,1,53.111111
2019-01-02 09:18:31.202200,1,59.041667
2019-01-02 09:20:54.195563,1,42.666667
